In [1]:
### Sri Balaji Muruganandam
### AAAMLP Book - Cross Validation chapter (Abhishek Thagur Book)
### K means Clustering and Validation
### Notebook - 2

In [2]:
"""
References
https://archive.ics.uci.edu/ml/datasets/wine+quality


"""

'\nReferences\nhttps://archive.ics.uci.edu/ml/datasets/wine+quality\n\n\n'

In [3]:
"""
cross-validation is a step in the process of building a machine learning model which
helps us ensure that our models fit the data accurately and also ensures that we do
not overfit.
"""

'\ncross-validation is a step in the process of building a machine learning model which\nhelps us ensure that our models fit the data accurately and also ensures that we do\nnot overfit.\n'

In [4]:
import numpy as np
import pandas as pd
import os
import inspect

In [5]:
### Taking the absolute path of input file
#dirname = os.path.dirname(__file__)
#input_file = os.path.join(dirname,"winequality-red.csv")

In [6]:
### To explain overfitting we are going to take red wine quality dataset
df = pd.read_csv("winequality-red.csv",sep=";",encoding="utf-8")

In [7]:
df.head(5)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [8]:
df.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [10]:
df.shape

(1599, 12)

In [11]:
## we need to find the quality of the wine based on other features
## if you see the values of quality column(our label) it is from 0 to 10

In [12]:
df.quality.to_numpy()[0:100]

array([5, 5, 5, 6, 5, 5, 5, 7, 7, 5, 5, 5, 5, 5, 5, 5, 7, 5, 4, 6, 6, 5,
       5, 5, 6, 5, 5, 5, 5, 6, 5, 6, 5, 6, 5, 6, 6, 7, 4, 5, 5, 4, 6, 5,
       5, 4, 5, 5, 5, 5, 5, 6, 6, 5, 6, 5, 5, 5, 5, 6, 5, 5, 7, 5, 5, 5,
       5, 5, 5, 6, 6, 5, 5, 4, 5, 5, 5, 6, 5, 4, 5, 5, 5, 5, 6, 5, 6, 5,
       5, 5, 5, 6, 5, 5, 4, 6, 5, 5, 5, 6])

In [13]:
## So we can take this problem as classification or regression.
## But we are taking it as a classification problem. Class 0 to 10
## Checking the unique quality classes

#df.quality.unique()
np.unique(df.quality.to_numpy())

array([3, 4, 5, 6, 7, 8])

In [14]:
pd.DataFrame(np.unique(df.quality.to_numpy(),return_counts=True))

,0,1,2,3,4,5
0,3,4,5,6,7,8
1,10,53,681,638,199,18


In [15]:
## The quality classes are mostly 5 and 6. The range is actually between 3 and 8
## So, we are mapping these classes as 0 to 5 from 3 to 8

map_values = {
    3:0,
    4:1,
    5:2,
    6:3,
    7:4,
    8:5
}

df.quality = df.quality.map(map_values)
df.head(5)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,2
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,2
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,2
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,3
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,2


In [16]:
df.quality.unique()

array([2, 3, 4, 1, 5, 0])

In [17]:
"""
When we look into the lables we can understand it is a classification problem. Generally this problem is 
solved with neural networks. We may have lot of classification algorithms in our head. 

Here, we are starting with decision tree model that we can visualize too
"""

'\nWhen we look into the lables we can understand it is a classification problem. Generally this problem is \nsolved with neural networks. We may have lot of classification algorithms in our head. \n\nHere, we are starting with decision tree model that we can visualize too\n'

In [18]:
### Just remember, we are doing this to understand what overfitting is
### Dividing the data into train and test data.
df.shape

(1599, 12)

In [19]:
### Shuffling the dataset
df.sample(frac=1)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
1388,6.6,0.640,0.31,6.1,0.083,7.0,49.0,0.99718,3.35,0.68,10.3,2
1061,9.1,0.400,0.50,1.8,0.071,7.0,16.0,0.99462,3.21,0.69,12.5,5
1342,7.6,0.540,0.02,1.7,0.085,17.0,31.0,0.99589,3.37,0.51,10.4,3
1572,7.3,0.690,0.32,2.2,0.069,35.0,104.0,0.99632,3.33,0.51,9.5,2
1153,9.6,0.420,0.35,2.1,0.083,17.0,38.0,0.99622,3.23,0.66,11.1,3
...,...,...,...,...,...,...,...,...,...,...,...,...
559,13.0,0.470,0.49,4.3,0.085,6.0,47.0,1.00210,3.30,0.68,12.7,3
718,8.4,0.560,0.04,2.0,0.082,10.0,22.0,0.99760,3.22,0.44,9.6,2
63,7.0,0.735,0.05,2.0,0.081,13.0,54.0,0.99660,3.39,0.57,9.8,2
734,7.7,0.560,0.20,2.0,0.075,9.0,39.0,0.99870,3.48,0.62,9.3,2


In [20]:
### Resetting the index after shuffling
df = df.sample(frac=1).reset_index(drop=True)
train_data = df.head(1000)
test_data = df.tail(599)

In [55]:
### Modelling a decision tree
from sklearn import tree
from sklearn import metrics

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

In [48]:
model = tree.DecisionTreeClassifier(max_depth=3, random_state=23)      ## Note Decision tree can be used for both classification and Regression

In [49]:
## Selecting all the columns except quality for X
columns = list(df.columns)
columns.remove("quality")

In [50]:
model.fit(train_data[columns],train_data.quality)

DecisionTreeClassifier(max_depth=3, random_state=23)

In [51]:
## We kept the max depth to 3 and keeping all other parameters as default
## Predicting
predict_train = model.predict(train_data[columns])
predict_test = model.predict(test_data[columns])

In [52]:
### Calculate the accuracy of train data
train_acc = metrics.accuracy_score(train_data.quality, predict_train)
train_acc

0.609

In [53]:
### Calculate the accuracy of test data
test_acc = metrics.accuracy_score(test_data.quality, predict_test)
test_acc

0.5575959933222037

In [64]:
### We can also use other metrics to calculate accuracy. We have used the basic one.
"""
Now, Experiementing with various depth values and plotting the accuracy for test and train data
"""

train_ls, test_ls = [],[]

for depth in range(1,30):
    model = tree.DecisionTreeClassifier(max_depth=depth, random_state=23)
    model.fit(train_data[columns],train_data.quality)
    
    predict_train = model.predict(train_data[columns])
    predict_test = model.predict(test_data[columns])
    
    train_acc = metrics.accuracy_score(train_data.quality, predict_train)
    test_acc = metrics.accuracy_score(test_data.quality, predict_test)
    
    train_ls.append(train_acc)
    test_ls.append(test_ls)

print("Training Acc", train_ls)

Training Acc [0.557, 0.557, 0.609, 0.628, 0.68, 0.725, 0.763, 0.806, 0.842, 0.872, 0.918, 0.949, 0.968, 0.977, 0.987, 0.991, 0.996, 0.998, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


In [ ]:
### Plotting both training and testing accuracy
### Setting label size
